In [17]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
tf.keras.utils.to_categorical

<function tensorflow.python.keras.utils.np_utils.to_categorical(y, num_classes=None, dtype='float32')>

In [26]:
train_data = pd.read_csv('classification.csv', dtype=str)
#data = pd.read_csv('./Hand_Annotations_2.csv',dtype=str,names=colnames
Y = train_data[['label']]

kf = KFold(n_splits = 10)
                         
skf = StratifiedKFold(n_splits = 10, random_state = 7, shuffle = True)
image_shape = (150,150,3)

In [22]:
idg = ImageDataGenerator(#width_shift_range=0.1,
                         #height_shift_range=0.1,
                         #zoom_range=0.3,
                         fill_mode='nearest',
                         horizontal_flip = True)
                         #rescale=1./255)

In [23]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [24]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, not binary!
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy', #not binary for the poses
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []

save_dir = '/kfoldModels/'
fold_var = 1

for train_index, val_index in kf.split(np.zeros(707),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]

    train_data_generator = idg.flow_from_dataframe(training_data, target_size=image_shape[:2],
                               x_col = "filename", y_col = "label",
                               class_mode = "categorical", shuffle = True)
    valid_data_generator  = idg.flow_from_dataframe(validation_data, target_size=image_shape[:2],
                            x_col = "filename", y_col = "label",
                            class_mode = "categorical", shuffle = True)
    
  

    # CREATE NEW MODEL
    #model = create_new_model()
    """result = model.fit_generator(train_data_generator,epochs=10,
                              steps_per_epoch=150,
                              validation_data=valid_data_generator,
                             validation_steps=12)
    model.save(save_dir+'/model_'+str(fold_var) +'.h5')"""
    # COMPILE NEW MODEL
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

    # CREATE CALLBACKS
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                            monitor='val_accuracy', verbose=1, 
                            save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    history = model.fit(train_data_generator,
                epochs=10,
                steps_per_epoch=150,
                callbacks=callbacks_list,
                validation_data=valid_data_generator)
    #PLOT HISTORY

    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights("/kfoldModels/model_"+str(fold_var)+".h5")

    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

    tf.keras.backend.clear_session()

    fold_var += 1



Found 636 validated image filenames belonging to 3 classes.
Found 71 validated image filenames belonging to 3 classes.
Epoch 1/10
150/150 [==============================] - ETA: 0s - loss: 0.3508 - accuracy: 0.9649